In [ ]:
# default_exp monte_carlo_shapley

In [ ]:
#export
# Author: Simon Grah <simon.grah@thalesgroup.com>
#         Vincent Thouvenot <vincent.thouvenot@thalesgroup.com>

# MIT License

# Copyright (c) 2020 Thales Six GTS France

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

In [ ]:
#export
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
#hide
np.random.seed(12345)

# Monte Carlo Shapley

> Estimate the Shapley Values using an optimized Monte Carlo version.

## Theory

### Shapley Value definition

In Collaborative Game Theory, Shapley Values ([Shapley,1953]) can distribute a reward among players in a fairly way according to their contribution to the win in a cooperative game. We note $\mathcal{M}$ a set of $d$ players. Moreover, $v : P(\mathcal{M}) \rightarrow R_v$ a reward function such that $v(\emptyset) = 0$. The range $R_v$ can be $\Re$ or a subset of $\Re$. $P(\mathcal{M})$ is a family of sets over $\mathcal{M}$. If $S \subset \mathcal{M}\text{, } v(S)$ is the amount of wealth produced by coalition $S$ when they cooperate.

The Shapley Value of a player $j$ is a fair share of the global wealth $v(\mathcal{M})$ produced by all players together:

$$\phi_j(\mathcal{M},v) = \sum_{S \subset \mathcal{M}\backslash \{j\}}\frac{(d -|S| - 1)!|S|!}{d!}\left(v(S\cup \{j\}) - v(S)\right),$$

with $|S| = \text{cardinal}(S)$, i.e. the number of players in coalition $S$.

### Shapley Values as contrastive local attribute importance in Machine Learning

Let be $X^*\subset\Re^d$ a dataset of individuals where a Machine Learning model $f$ is trained and/or tested and $d$  the dimension of $X^*$. $d>1$ else we do not need to compute Shapley Value. We consider the attribute importance of an individual $\mathbf{x^*} = \{x_1^*, \dots, x_d^*\} \in X^*$ according to a given reference $\mathbf{r} = \{r_1, \dots, r_d\}\in X^*$.  We're looking for $\boldsymbol{\phi}=(\phi_j)_{j\in\{1, \dots, d\}}\in \Re^d$ such that:
$$ \sum_{j=1}^{d} \phi_j = f(\mathbf{x^*}) - f(\mathbf{r}), $$ 
where $\phi_j$ is the attribute contribution of feature indexed $j$.  We loosely identify each feature by its column number. Here the set of players $\mathcal{M}=\{1, \dots, d\}$ is the feature set.

In Machine Learning, a common choice for the reward is $ v(S) = \mathbb{E}[f(X) | X_S = \mathbf{x_S^*}]$, where $\mathbf{x_S^*}=(x_j^*)_{j\in S}$ and $X_S$ the element of $X$ for the coalition $S$. 
For any $S\subset\mathcal{M}$, let's define $ z(\mathbf{x^*},\mathbf{r},S)$ such that $z(\mathbf{x^*},\mathbf{r},\emptyset) = \mathbf{r}$, \ $z(\mathbf{x^*},\mathbf{r},\mathcal{M}) = \mathbf{x^*}$ and

$$ z(\mathbf{x^*},\mathbf{r},S) = (z_1,\dots, z_d) \text{ with } z_i =  x_i^* \text{ if } i \in S \text{ and } r_i  \text{ otherwise }$$ 

As explain in [Merrick,2019], each reference $\textbf{r}$ sets a single-game with $ v(S) = f(z(\mathbf{x^*},\mathbf{r},S)) - f(\mathbf{r}) $, $v(\emptyset) = 0 $ and $v(\mathcal{M}) = f(\mathbf{x^*}) - f(\mathbf{r}) $.

Furthermore, we can extend the previous result by using several references well chosen. In that case, the final Shapley Values obtained are simply the average of those calculated on each reference independantly. But, in order to accelerate the estimation, we modify the algorithm to take into account this situation.  

### Optimized Monte Carlo Algorithm

**Inputs:** instance $\mathbf{x^*}$ and $\mathbf{r}$, the reward function $v$ and the number of iterations $\text{T}$.

**Result:** the Shapley Values $\boldsymbol{\widehat{\phi}} \in \Re^d$.

1.&emsp;Initialization: $\boldsymbol{\widehat{\phi}}=  \{0,\dots,0\}$ \;

2.&emsp;For $t=1,\dots,T$:<br>
&emsp;&emsp;(a).&emsp;Choose the subset resulting of an uniform permutation $O\in\pi(\{1, \dots,d\})$ of the features values \;<br>
&emsp;&emsp;(b).&emsp;If several references are given, select at random one reference $\mathbf{r}$ \;<br>
&emsp;&emsp;(c).&emsp; $v^{(1)} = v(\mathbf{r})$, $\mathbf{b} = \mathbf{r}$ \;<br>
&emsp;&emsp;(d).&emsp; For j in $O$:<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\mathbf{b} = x_i^* \text{ if } i = j \text{ and } b_i \text{ otherwise}$, with $i\in\{1, \dots, d\}$ \;<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$ v^{(2)} = v(\mathbf{b})$ \;<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\phi_j =  v^{(2)} - v^{(1)}$ \;<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Update online $\widehat{\phi}$ (if $t > 1$):<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\widehat{\phi_j} = \dfrac{t-1}{t} \widehat{\phi_j} + \dfrac{1}{t} \phi_j$ \;<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$v^{(1)} = v^{(2)}$ \;<br>

### References

[Shapley,1953] _A value for n-person games_. Lloyd S Shapley. In Contributions to the Theory of Games, 2.28 (1953), pp. 307 - 317.

[Merrick,2019] _The Explanation Game: Explaining Machine Learning Models with Cooperative Game Theory_. Luke Merrick, Ankur Taly, 2019.

## Function 

__Parameters__

* `x`: pandas Series. The instance $\mathbf{x^*}$ for which we want to calculate Shapley value of each attribute,

* `fc`: python function. The reward function $v$,

* `ref`: pandas Series or pandas DataFrame. Either one or several references $\mathbf{r}$. The Shapley values (attribute importance) is a contrastive explanation according to these individual(s),

* `n_iter`: integer. The number of iteration, 

* `callback`: An python object which can be called at each iteration to record distance to minimum for example. At each iteration, callback(Φ)

__Returns__

* `Φ`: pandas Series. Shapley values of each attribute

In [ ]:
#export
def MonteCarloShapley(x, fc, ref, n_iter, callback=None, seed=0):
    """
    Estimate the Shapley Values using an optimized Monte Carlo version.
    """
    
    np.random.seed(seed)

    # Get general information
    feature_names = list(x.index)
    d = len(feature_names) # dimension

    # Individual reference or dataset of references
    if isinstance(ref, pd.core.series.Series):
        individual_ref = True
        f_r = fc(ref.values)
    elif isinstance(ref, pd.core.frame.DataFrame):
        if ref.shape[0] == 1:
            ref = ref.iloc[0]
            individual_ref = True
            f_r = fc(ref.values)
        else:
            individual_ref = False
            n_ref = len(ref)

    if individual_ref:
        # If x[j] = r[j] => Φ[j] = 0 and we can reduce the dimension
        distinct_feature_names = list(x[x!=ref].index)
        if set(distinct_feature_names) == set(feature_names):
            distinct_feature_names = feature_names
            sub_d = d
            x_cp = x.copy()
            r_cp = ref.copy()
            reward = lambda z: fc(z)
            pass
        else:
            sub_d = len(distinct_feature_names) # new dimension
            x_cp = x[distinct_feature_names].copy()
            r_cp = ref[distinct_feature_names].copy()
            print("new dimension {0}".format(sub_d))
            def reward(z):
                z_tmp = ref.copy()
                z_tmp[distinct_feature_names] = z
                return fc(z_tmp.values)
    else:
        distinct_feature_names = feature_names
        sub_d = d
        x_cp = x.copy()
        reward = lambda z: fc(z)

    # Store all Shapley Values in a numpy array
    Φ_storage = np.empty((n_iter, sub_d))

    # Monte Carlo loop
    for m in tqdm(range(1, n_iter+1)):
        # Sample a random permutation order
        o = np.random.permutation(sub_d)
        # initiate useful variables for this iteration
        # if several references select at random one new ref at each iter
        if individual_ref:
            f_less_j = f_r
            x_plus_j = r_cp.values.copy()
        else:
            r_cp = ref.values[np.random.choice(n_ref, size=1)[0],:].copy()
            f_less_j = fc(r_cp)
            x_plus_j = r_cp.copy()
        # iterate through the permutation of features
        for j in o:
            x_plus_j[j] = x_cp.values[j]
            f_plus_j = reward(x_plus_j)
            # update Φ
            Φ_j = f_plus_j - f_less_j
            Φ_storage[m-1,j] = Φ_j
            # reassign f_less_j
            f_less_j = f_plus_j
        if callback:
            Φ = pd.Series(np.mean(Φ_storage[:m,:],axis=0), index=feature_names)
            callback(Φ)

    Φ_mean = np.mean(Φ_storage,axis=0)
    Φ = pd.Series(np.zeros(d), index=feature_names)
    Φ[distinct_feature_names] = Φ_mean
    return Φ

In [ ]:
#export
def MonteCarloShapleyBatch_firstversion(x, fc, ref, n_iter, seed=0):
    """
    Estimate the Shapley Values using an optimized Monte Carlo version in Batch mode.
    """
    
    np.random.seed(seed)

    # Get general information
    feature_names = list(x.index)
    dimension = len(feature_names)

    # Individual reference or dataset of references
    if isinstance(ref, pd.core.series.Series):
        individual_ref = True
    elif isinstance(ref, pd.core.frame.DataFrame):
        if ref.shape[0] == 1:
            ref = ref.iloc[0]
            individual_ref = True
        else:
            individual_ref = False
            n_ref = len(ref)

    # Compute the matrix X of hybrid individuals between x and ref
    # and keep trace of permutation orders
    dtype_features = object if x.dtype == object else float
    array_of_hybrid_individuals = np.empty(shape=(n_iter * (dimension+1), dimension), dtype=dtype_features)
    orders = np.zeros(shape=(n_iter * (dimension+1),), dtype=np.int32)

    for iter_monte_carlo in range(n_iter):
        order = np.random.permutation(dimension)
        if individual_ref == True:
            hybrid_individual = ref.values.copy()
        else:
            hybrid_individual = ref.values[np.random.choice(n_ref, size=1)[0],:].copy()
        array_of_hybrid_individuals[iter_monte_carlo * (dimension+1),:] = hybrid_individual
        orders[iter_monte_carlo * (dimension+1)] = -1
        for iter_order, idx_feature in enumerate(order):
            hybrid_individual[idx_feature] = x.values[idx_feature]
            array_of_hybrid_individuals[iter_monte_carlo*(dimension+1) + (iter_order+1),:] = hybrid_individual
            orders[iter_monte_carlo*(dimension+1) + (iter_order+1)] = idx_feature

    try:
        rewards = fc(array_of_hybrid_individuals)
    except Exception as e:
        print("Oops!", e.__class__, "occurred.")
        print("Your function fc should be able to handle a data set of inputs")
        exit()

    rewards_diff = np.diff(rewards)

    mc_shap_batch = pd.Series(np.zeros(dimension), index=feature_names)
    for idx_feature in range(dimension):
        shap_val_feature = np.mean(rewards_diff[orders[1:] == idx_feature])
        mc_shap_batch[idx_feature] = shap_val_feature

    return mc_shap_batch

In [ ]:
#export
def MonteCarloShapleyBatch(x, fc, ref, n_iter, seed=0):
    """
    Estimate the Shapley Values using an optimized Monte Carlo version in Batch mode.
    """
    np.random.seed(seed)

    # Get general information
    feature_names = list(x.index)
    dimension = len(feature_names)
    # Individual reference or dataset of references
    if isinstance(ref, pd.core.series.Series):
        individual_ref = True
    elif isinstance(ref, pd.core.frame.DataFrame):
        if ref.shape[0] == 1:
            ref = ref.iloc[0]
            individual_ref = True
        else:
            individual_ref = False
            n_ref = len(ref)
    
    # Compute the matrix X of hybrid individuals between x and ref
    # and keep trace of permutation orders
    dtype_features = object if x.dtype == object else float
    array_of_hybrid_individuals = []
    orders = []
    for iter_monte_carlo in range(n_iter):
        order = np.random.permutation(dimension)
        if individual_ref == True:
            hybrid_individual = ref.values.copy()
        else:
            hybrid_individual = ref.values[np.random.choice(n_ref, size=1)[0],:].copy()
        array_of_hybrid_individuals.append(np.copy(hybrid_individual))
        orders.append(-1)
        for iter_order, idx_feature in enumerate(order):
            hybrid_individual[idx_feature] = x.values[idx_feature]
            array_of_hybrid_individuals.append(np.copy(hybrid_individual))
            orders.append(idx_feature)       
    array_of_hybrid_individuals = np.array(array_of_hybrid_individuals)
    orders = np.array(orders)
    try:
        rewards = fc(array_of_hybrid_individuals)
    except Exception as e:
        print("Oops!", e.__class__, "occurred.")
        print("Your function fc should be able to handle a data set of inputs")
        exit()

    rewards_diff = np.diff(rewards)
    mc_shap_batch = pd.Series(np.zeros(dimension), index=feature_names)
    for idx_feature in range(dimension):
        shap_val_feature = np.mean(rewards_diff[orders[1:] == idx_feature])
        mc_shap_batch[idx_feature] = shap_val_feature

    return mc_shap_batch

## Example

We use a simulated dataset from the book _Elements of Statistical Learning_ ([hastie,2009], the Radial example). $X_1, \dots , X_{d}$ are standard independent Gaussian. The model is determined by:

$$ Y = \prod_{j=1}^{d} \rho(X_j), $$

where $\rho\text{: } t \rightarrow \sqrt{(0.5 \pi)} \exp(- t^2 /2)$. The regression function $f_{regr}$ is deterministic and simply defined by $f_r\text{: } \textbf{x} \rightarrow \prod_{j=1}^{d} \phi(x_j)$. For a reference $\mathbf{r^*}$ and a target $\mathbf{x^*}$, we define the reward function $v_r^{\mathbf{r^*}, \mathbf{x^*}}$ such as for each coalition $S$, $v_r^{\mathbf{r^*}, \mathbf{x^*}}(S) = f_{regr}(\mathbf{z}(\mathbf{x^*}, \mathbf{r^*}, S)) - f_{regr}(\mathbf{r^*}).$

 [hastie,2009] _The Elements of Statistical Learning: Data Mining, Inference, and Prediction, Second Edition_. Hastie, Trevor and Tibshirani, Robert and Friedman, Jerome. Springer Series in Statistics, 2009.
	

In [ ]:
d, n_samples = 5, 100
mu = np.zeros(d)
Sigma = np.zeros((d,d))
np.fill_diagonal(Sigma, [1] * d)
X = np.random.multivariate_normal(mean=mu, cov=Sigma, size=n_samples)
X = pd.DataFrame(X, columns=['x'+str(i) for i in range(1, d+1)])
def fc(x):
    phi_x = np.sqrt(.5 * np.pi) * np.exp(-0.5 * x ** 2)
    return np.prod(phi_x)
def fc_batch(X):
    phi_x = np.sqrt(.5 * np.pi) * np.exp(-0.5 * X ** 2)
    return np.prod(phi_x, axis=1)
y = fc_batch(X.values)
n = 2**d - 2
print("dimension = {0} ; nb of coalitions = {1}".format(str(d), str(n)))

dimension = 5 ; nb of coalitions = 30


### Pick an individual x to explain

In [ ]:
x = X.iloc[np.random.choice(len(X), size=1)[0],:]
x

x1    1.747234
x2   -1.410246
x3   -0.378242
x4   -0.345821
x5    0.380062
Name: 60, dtype: float64

### Single reference

In [ ]:
reference = X.iloc[np.random.choice(len(X), size=1)[0],:]
reference

x1    0.081577
x2   -0.302335
x3   -0.726916
x4    0.180335
x5   -0.520209
Name: 94, dtype: float64

In [ ]:
mc_shap = MonteCarloShapley(x=x, fc=fc, ref=reference, n_iter=10000, callback=None)

100%|██████████| 10000/10000 [00:01<00:00, 8659.86it/s]


In [ ]:
mc_shap

x1   -1.162240
x2   -0.791358
x3    0.192819
x4   -0.041533
x5    0.062293
dtype: float64

In [ ]:
mc_shap_batch1 = MonteCarloShapleyBatch_firstversion(x=x, fc=fc_batch, ref=reference, n_iter=10000)

In [ ]:
mc_shap_batch1

x1   -1.162240
x2   -0.791358
x3    0.192819
x4   -0.041533
x5    0.062293
dtype: float64

In [ ]:
mc_shap_batch = MonteCarloShapleyBatch(x=x, fc=fc_batch, ref=reference, n_iter=10000)

In [ ]:
mc_shap_batch

x1   -1.162240
x2   -0.791358
x3    0.192819
x4   -0.041533
x5    0.062293
dtype: float64

### Several references 

In [ ]:
references = X.iloc[np.random.choice(len(X), size=10, replace=False),:]
references

,x1,x2,x3,x4,x5
12,0.862580,-0.010032,0.050009,0.670216,0.852965
32,-0.416232,-0.116747,-1.844788,2.068708,-0.776967
33,1.440167,-0.110557,1.227387,1.920784,0.746433
94,0.081577,-0.302335,-0.726916,0.180335,-0.520209
17,1.824875,-0.997518,0.850591,-0.131578,0.912414
97,-1.872339,-0.962791,0.080067,0.128726,-0.479120
2,1.007189,-1.296221,0.274992,0.228913,1.352917
99,-0.056715,2.300675,0.569497,1.489410,1.264250
64,-0.105434,0.700428,2.092852,-0.136972,-0.930489
36,-0.567455,-0.372642,-0.926557,1.755108,1.209810


In [ ]:
mc_shaps = MonteCarloShapley(x=x, fc=fc, ref=references, n_iter=10000, callback=None)

100%|██████████| 10000/10000 [00:03<00:00, 3251.37it/s]


In [ ]:
mc_shaps

x1   -0.359249
x2   -0.243412
x3    0.136020
x4    0.119385
x5    0.108600
dtype: float64

In [ ]:
mc_shaps_batch1 = MonteCarloShapleyBatch_firstversion(x=x, fc=fc_batch, ref=references, n_iter=10000)

In [ ]:
mc_shaps_batch1

x1   -0.359249
x2   -0.243412
x3    0.136020
x4    0.119385
x5    0.108600
dtype: float64

In [ ]:
mc_shaps_batch = MonteCarloShapleyBatch(x=x, fc=fc_batch, ref=references, n_iter=10000)

In [ ]:
mc_shaps_batch

x1   -0.359249
x2   -0.243412
x3    0.136020
x4    0.119385
x5    0.108600
dtype: float64

In [ ]:
#hide
from shapkit.shapley_values import ShapleyValues

## Tests

In [ ]:
x_pred = fc(x.values)
reference_pred = fc(reference.values)
fcs = []
for r in references.values:
    fcs.append(fc(r))
references_pred = np.mean(fcs)

In [ ]:
assert np.abs(mc_shap.sum() - (x_pred - reference_pred)) <= 1e-10 

In [ ]:
assert np.abs(mc_shap_batch1.sum() - (x_pred - reference_pred)) <= 1e-10 

In [ ]:
assert np.abs(mc_shap_batch.sum() - (x_pred - reference_pred)) <= 1e-10 

In [ ]:
assert np.abs(mc_shaps.sum() - (x_pred - references_pred)) <= 1e-2

In [ ]:
assert np.abs(mc_shaps_batch1.sum() - (x_pred - references_pred)) <= 1e-2

In [ ]:
assert np.abs(mc_shaps_batch.sum() - (x_pred - references_pred)) <= 1e-2

In [ ]:
true_shap = ShapleyValues(x=x, fc=fc, ref=reference)
assert np.linalg.norm(mc_shap - true_shap, 2) <= 1e-2

100%|██████████| 5/5 [00:00<00:00, 240.72it/s]


In [ ]:
assert np.linalg.norm(mc_shap_batch - true_shap, 2) <= 1e-2

In [ ]:
true_shaps = ShapleyValues(x=x, fc=fc, ref=references)
assert np.linalg.norm(mc_shaps - true_shaps, 2) <= 1e-2

100%|██████████| 5/5 [00:00<00:00, 48.49it/s]


In [ ]:
np.linalg.norm(mc_shaps - true_shaps, 2)

0.007806901849042836

In [ ]:
assert np.linalg.norm(mc_shaps_batch1 - true_shaps, 2) <= 1e-2

In [ ]:
assert np.linalg.norm(mc_shaps_batch - true_shaps, 2) <= 1e-2

## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted index.ipynb.
Converted inspector.ipynb.
Converted monte_carlo_shapley.ipynb.
Converted plots.ipynb.
Converted sgd_shapley.ipynb.
Converted shapley_values.ipynb.
